In [ ]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks")

In [ ]:
!ls

# load メルカリデータ

In [ ]:
csv = "./data/mercari_ルンバ_202210231444.csv"
df = pd.read_csv(csv)
df.columns = ["name", "price", "url"]
df = df[["name", "price"]]

In [ ]:
import unicodedata

# df["name_org"] = df["name"]
# df["name"] = unicodedata.normalize("NFKC", df["name"])

df["name"] = df["name"].apply(lambda x: unicodedata.normalize("NFKC",x))
df["junk"] = df["name"].str.contains("ジャンク")
df


In [ ]:
# import re

# import unicodedata
# s = "１２３①②③,㍻㍉㌦㎡㈱№,ａｂｃＡＢＣ,Å🅱©,123abcABC,＜＞,アイウエオｶｷｸｹｺ"
# unicodedata.normalize("NFKC", s)

# power_words = ["irobot", "iroboot", "アイロボット", "ルンバ", "roomba", "リモコン", "交換部品", "付属品"]
# power_words += ["バッテリー", "バーチャルウォール", "フィルター", "ブラシ"]
# power_words += [
#     "i3", "i7", "s9", "e5", "j7",
#     "500シリーズ", "600シリーズ", "700シリーズ", "800シリーズ",
# ]
# power_words += ["中古", "ジャンク"]

# drop_words = ["(", ")", "[", "]", "［", "］", "/",
#                 "、", "。", "【", "】", "の画像" ]


# pattern = r'^(0|[1-9]\d*|[1-9]\d{0,2}(,\d{3})+)円'


# keywords = []
# for r in df.name:
#     r = unicodedata.normalize("NFKC", r)
#     r = r.lower()

#     for w in power_words:
#         r = r.replace(w, f" {w} ")

#     for w in drop_words:
#         r = r.replace(w, " ")
    
#     r = re.sub(r'[\d,]+円', "0円", r) # 1,xxx円

#     ws = [s.strip() for s in r.split()]

#     # XX年
#     for i in range(len(ws)):

#         s = re.findall(r'^1[0-9]年', ws[i])
#         if len(s)>0:
#             ws[i] = f" 20{ws[i][:2]}年 <-{ws[i]}(1) "

#         # XX年
#         r = ws[i]
#         finds = re.findall(r'20[0-3][0-9]年', r)
#         if len(finds)>0:
#             ws[i] = f" {finds[0]} <-{r}(2)"



#     keywords += ws

# sr_keywords = pd.Series(keywords).drop_duplicates().sort_values()
# # .unique().tolist()

# sr_keywords.to_csv("./data/mercari_ルンバ_202210231444_keywords.csv", index=False)

In [ ]:
import re
class ModelYear:
    def __init__(self, year_min, year_max):
        self.min = year_min
        self.max = year_max

    def parse(self, s):

        years = []

        ss = re.findall(r'20[0-1][0-9]年', s)
        if len(ss)>0:
            year = ss[0].replace("年", "")
            years.append(year)

        ss = re.findall(r'202[0-2]年', s)
        if len(ss)>0:
            year = ss[0].replace("年", "")
            years.append(year)

        years = [int(y) for y in years]
        years = [ y for y in years if self.min <= y <= self.max]

        if len(years)>0:
            year = min(years)
            return int(year)

    @staticmethod
    def _test():
        my = ModelYear(2000, 2022)
        est = my.parse("IROBOT  2021年最新ロボット掃除機ルンバ 2009  i3＋ i3550の画像 80,000円")
        assert int(est) == 2021, est

ModelYear._test()

In [ ]:
df["year"] = df["name"].apply(ModelYear(2005, 2022).parse)
df

In [ ]:
class ModelNumber:

    def parse(self, s):

        # 金額除去
        s = re.sub(r'[\d,]+円', "0円", s) # 1,xxx円
        
        # 数字だけ取り出し
        bufs = []
        nums = re.findall(r"\d+", s)
        for num in nums:
            
            if num in ["500", "600", "700", "800", "900"]:
                continue

            if len(num) < 3:
                continue
            
            if len(num) == 3:
                bufs.append(num)
                continue
            
            if len(num) == 6:
                num = num.replace("060", "")
                bufs.append(num)
                continue
        
        bufs = [b for b in bufs if int(b) >= 500]

        bufs = list(set(bufs))

        if len(bufs)>0:
            return bufs[0]
        else:
            return None


    @staticmethod
    def _test():
        mn = ModelNumber()
        est = mn.parse("ルンバ i7 ロボット掃除機 アイロボット i715060【新品・未開封】の画像 46,300円	")
        assert est == "715", est

# ModelNumber._test()

In [ ]:
df["model"] = df["name"].apply(ModelNumber().parse)
df["mid_model"] = df["model"].apply(lambda x: int(int(x[:2])*10) if x is not None else None)
df["big_model"] = df["model"].apply(lambda x: int(int(x[:1])*100) if x is not None else None)


df.to_csv("df_model.csv", index=False)

df.sample(20)


In [ ]:
# yearチェック
dfp = df.sort_values("year").dropna(subset=["year"])

dfp.to_csv("df.csv", index=False)

dfp

# カシカ

In [ ]:
ax = plt.subplots(figsize=(12, 4))
ax = sns.boxplot(x="year", y="price", data=df,
            whis=[0, 100], width=.9, palette="vlag")

sns.stripplot(x="year", y="price", data=df, 
              size=4, 
              color=".3",
               linewidth=0)

ax.xaxis.grid(True)
sns.despine(trim=True, left=True)



In [ ]:
ax = plt.subplots(figsize=(30, 8))
ax = sns.boxplot(x="model", y="price", data=df.sort_values("model"),
            whis=[0, 100], width=.9, palette="vlag")

sns.stripplot(x="model", y="price", data=df.sort_values("model"),
              size=4, 
              color=".3",
               linewidth=0)

ax.xaxis.grid(True)
sns.despine(trim=True, left=True)

plt.title("roomba")

plt.ylim(0, 80000)


In [ ]:
ax = plt.subplots(figsize=(20, 8))
ax = sns.boxplot(x="mid_model", y="price", data=df,
            whis=[0, 100], width=.9, palette="vlag")

sns.stripplot(x="mid_model", y="price", data=df, 
              size=4, 
              color=".3",
               linewidth=0)

ax.xaxis.grid(True)
sns.despine(trim=True, left=True)

plt.title("roomba")


In [ ]:
ax = plt.subplots(figsize=(20, 8))
m = df["junk"] == False
ax = sns.boxplot(x="mid_model", y="price", data=df[m],
            whis=[0, 100], width=.9, palette="vlag")

sns.stripplot(x="mid_model", y="price", data=df[m], 
              size=4, hue="junk",
              color=".3",
               linewidth=0)

ax.xaxis.grid(True)
sns.despine(trim=True, left=True)

plt.title("roomba")


In [ ]:
ax = plt.subplots(figsize=(15, 8))
ax = sns.boxplot(x="big_model", y="price", data=df,
            whis=[0, 100], width=.9, palette="vlag")

sns.stripplot(x="big_model", y="price", data=df, 
              size=4, 
              color=".3",
               linewidth=0)

ax.xaxis.grid(True)
sns.despine(trim=True, left=True)

plt.title("roomba")


In [ ]:
ax = plt.subplots(figsize=(15, 8))
ax = sns.boxplot(x="year", y="price", data=df,
            whis=[0, 100], width=.9, palette="vlag")

sns.stripplot(x="year", y="price", data=df, 
              size=4, 
              color=".3",
               linewidth=0)

ax.xaxis.grid(True)
sns.despine(trim=True, left=True)

plt.title("roomba")


In [ ]:
m = (df["mid_model"] == 760) | (df["mid_model"] == 770)


df[m].sort_values("price")

# Visualization

In [ ]:
plt.figure(figsize=(10, 5))
df.price.hist(bins=100)
_ = plt.title(csv)